In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

login_url = 'https://www.jobplanet.co.kr/users/sign_in'

#email 본인 아이디, password 본인 패스워드 입력 단, 리뷰를 남겨서 전체 접근이 가능한 상태여야함
email = 'wondh07@naver.com'
password = 'xxxx'


LOGIN_INFO ={
    'user[email]' : email,
    'user[password]' : password,
    'commit' : '로그인'
}

session = requests.session()

res = session.post(login_url, data = LOGIN_INFO, verify = False)

res

res.raise_for_status()
result = []


def clean_str(text):
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    text = text.replace('\r','. ')
    return text

#url 은 보고싶은 기업의 리뷰 URL 이며 마지막은 ?page= 형태로 해야함, last_page는 해당 기업 리뷰의 마지막 페이지 입력
last_page = 3
for idx in range(1,last_page):
    url = 'https://www.jobplanet.co.kr/companies/1322/reviews/현대엔지니어링?page='+str(idx)

    res = session.get(url)
    res.raise_for_status()

    soup = BeautifulSoup(res.text, 'html.parser')

    count3 = 0
    count4 = 0
    count5 = 0

    try:
        for k in range(5):
            reviewer_info = []
            # 응답자 정보
            position = soup.select('.content_top_ty2 > span.txt1')[0 + count4].text
            status = soup.select('.content_top_ty2 > span.txt1')[1 + count4].text
            loc = soup.select('.content_top_ty2 > span.txt1')[2 + count4].text

            day = soup.select('.content_top_ty2 > span.txt1')[3+ count4].text

            # 점
            star_rating = soup.select('.us_star_m > div.star_score')[0+k]['style'][6:-1]

            # rating 5*5
            promotion = soup.select('.bl_score')[0 + count5]['style'][6:-1]
            welfare = soup.select('.bl_score')[1 + count5]['style'][6:-1]
            balance = soup.select('.bl_score')[2 + count5]['style'][6:-1]
            culture = soup.select('.bl_score')[3 + count5]['style'][6:-1]
            top = soup.select('.bl_score')[4 + count5]['style'][6:-1]
            # 중심 제목
            content = soup.select('h2.us_label')[0+k].text
            # 장단점 경영진 의견
            merit = soup.select('dl.tc_list > dd.df1 > span')[0 + count3].text
            disadvantages = soup.select('dl.tc_list > dd.df1 > span')[1 + count3].text
            df_tit = soup.select('dl.tc_list > dd.df1 > span')[2 + count3].text


            reviewer_info = [position, status, loc, day, star_rating, promotion, welfare, balance, culture, top,clean_str(content),clean_str(merit),clean_str(disadvantages), clean_str(df_tit)]

            result.append(reviewer_info)
            reviewer_info=[]
            count3 += 3
            count4 += 4
            count5 += 5
            print("pass :"+str(idx)+"-"+str(k))
    except :
        print("fail :" + str(idx))
        pass

colname = ['직무','상황','지역','작성일','총점','승진 기회 및 가능성','복지 및 급여','업무와 삶의 균형','사내문화','경영진','총평','장점','단점','바라는점']
df = pd.DataFrame(result,columns=colname)
#저장을 희망하는 파일명으로 저장
#df.to_excel("jobplanet_기업명.xlsx")


print(result)

print(df)



/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.jobplanet.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.jobplanet.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


pass :1-0
pass :1-1
pass :1-2
pass :1-3
pass :1-4


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.jobplanet.co.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


pass :2-0
pass :2-1
pass :2-2
pass :2-3
pass :2-4
[['금융/재무', '\n          전직원\n      ', '서울', '2023. 09', '60%', '60%', '100%', '80%', '60%', '60%', '\nBEST\n      한국에서 이정도면 괜찮은 회사인 편이라고 생각합니다\n    ', '서울 도심에 사무실 있는거 좋고 복지 괜찮고 사람들도 대체로 좋은 사람들 많음', '바뀌려고 노력하고 있지만 여전히 딱딱하고 보수적인 분위기가 답답함', '권위적이고 수직적인 분위기를 타파하는것이 회사 발전에 도움이 될것 같습니다'], ['엔지니어링', '\n          현직원\n      ', '부산', '2023. 09', '60%', '60%', '60%', '40%', '60%', '40%', '\nBEST\n      경력직이 너무 많은 회사 회사 고유의 색깔이 뚜렷하지 않아서 어느현장이냐에 따라 다 업무스타일이 다름\n    ', '경력직이 많아 소위 꼰대가 랜덤하게 있음 샌드위치 데이에는 단체 강제연차 사용', '경력직이 너무 많음 회사 고유의 색깔이 부족함 그래서 현장마다 스타일이 다 다름', '경력직 많이 넣는것도 좋지만 회사의 색깔을 잃어버리지 않을려면 최소 현엔 시스템에대해 자세히 아는사람이 많아야됨'], ['엔지니어링', '\n          전직원\n      ', '서울', '2023. 09', '80%', '60%', '60%', '80%', '80%', '80%', '\nBEST\n      좋은 회사라고 생각합니다 전반적으로 만족한 회사에요\n    ', '계약직이어서 복지는 거의 해당이 없었지만 그래도 대기업인 만큼 사람들도 괜찮고 분위기도 나랑 잘 맞는 편이었던듯', '급여가 좀 적고 남녀성비가 안맞는게 단점이라면 단점이지만 나랑은 잘 맞았음', '바라는 점은 딱히 없습니다 좋은 분위기를 가지고 있다고 생각해요'], ['법률/법무', '\n          현직원\

In [10]:
df.to_csv('SaveCSV2.csv', encoding='utf-8-sig',index=False, header=False)

from google.colab import files

files.download('SaveCSV2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>